# eoAPI Demo: Analyzing Maxar Data on Hawaii Wildfires

### Background

The Hawaii wildfires have been devastating, causing significant loss of life and property. This crisis is a stark reminder of the importance of timely and accurate data for disaster response.

### Objective

This notebook aims to demonstrate how [eoAPI](https://github.com/developmentseed/eoAPI) can be used to analyze Maxar's high-resolution satellite data for assessing the impact of the Hawaii wildfires of August 2023.

### Maxar Open Data

Pre and post-event high-resolution satellite imagery in support of emergency planning, risk assessment, monitoring of staging areas and emergency response, damage assessment, and recovery. These images are generated using the Maxar ARD pipeline, tiled on an organized grid in analysis-ready cloud-optimized formats.

### STAC and COGs

Maxar releases open data for select sudden-onset major crisis events. In addition to making the data (as nice COGs) freely available on AWS, they also add STAC (static) metadata alongside the images. Having the STAC items already created makes ingestion into the PgSTAC database easy (we don't have to produce the items ourselves and thus have to read the images).

To learn more about how we ingest the Maxar OpenData STAC catalog into PgSTAC see https://github.com/vincentsarago/MAXAR_opendata_to_pgstac.

---

> 📣 **Note: Important Information**
> - **Demo Endpoints**: This notebook uses the demo endpoints [stac.eoapi.dev](https://stac.eoapi.dev) and [raster.eoapi.dev](https://raster.eoapi.dev). These are not intended for production use.
> - **Authentication**: While the demo endpoints are currently open to the public, we may introduce authentication features in the future.
> - **Set Up Your Own Services**: If you're interested in setting up your own eoAPI services, you can easily do so. For a step-by-step guide, please check out the 'Getting Started' section of the [eoAPI GitHub repository](https://github.com/developmentseed/eoAPI).
> - **Breaking changes**: DevelopmentSeed does not guarantee that these eoAPI endpoints will stay `open`, and breaking changes might be introduced anytime.

---

# STAC Metadata

Endpoint: https://stac.eoapi.dev

In [1]:
# Requirements

# !python -m pip install httpx folium

In [2]:
import json
import httpx

from folium import Map, TileLayer, GeoJson

## Collections


In this section, we will:

- List all the collections stored in the pgSTAC database (backend of the eoapi STAC service
- Talk about the collection metadata (and extensions)
- See how to visualize the collections on a Map using the collection's Spatial Extent 

In [3]:
# Let's see what are the collections available in `stac.eoapi.dev` instance
collections = httpx.get("https://stac.eoapi.dev/collections").json()
print(f"Found {len(collections['collections'])} Collections")

Found 29 Collections


In [4]:
# Print the names 
print("Collection names:")
print([c["id"] for c in collections["collections"]])

Collection names:
['MAXAR_BayofBengal_Cyclone_Mocha_May_23', 'MAXAR_Emilia_Romagna_Italy_flooding_may23', 'MAXAR_Gambia_flooding_8_11_2022', 'MAXAR_Marshall_Fire_21_Update', 'MAXAR_Hurricane_Fiona_9_19_2022', 'MAXAR_Hurricane_Ian_9_26_2022', 'MAXAR_Indonesia_Earthquake22', 'MAXAR_Kahramanmaras_turkey_earthquake_23', 'MAXAR_Kalehe_DRC_Flooding_5_8_23', 'MAXAR_volcano_indonesia21', 'MAXAR_New_Zealand_Flooding22', 'MAXAR_New_Zealand_Flooding23', 'MAXAR_Sudan_flooding_8_22_2022', 'MAXAR_afghanistan_earthquake22', 'MAXAR_cyclone_emnati22', 'MAXAR_ghana_explosion22', 'MAXAR_kentucky_flooding_7_29_2022', 'MAXAR_pakistan_flooding22', 'MAXAR_southafrica_flooding22', 'MAXAR_tonga_volcano21', 'MAXAR_yellowstone_flooding22', 'MAXAR_Maui_Hawaii_fires_Aug_23', 'MAXAR_NWT_Canada_Aug_23', 'MAXAR_shovi_georgia_landslide_8Aug23', 'MAXAR_Hurricane_Idalia_Florida_Aug23', 'MAXAR_Libya_Floods_Sept_2023', 'MAXAR_McDougallCreekWildfire_BC_Canada_Aug_23', 'MAXAR_Morocco_Earthquake_Sept_2023', 'UMBRA_2023']


Each collection should be in the form:
```json
{
    "id": ...,
    "type": "Collection",
    "links": [...],
    "extent": {
        "spatial": [
            [...],  // combined bbox,
            [...],  // bbox 1,
            ...
            [...],  // bbox n,
        ],
        "temporal": {
            "interval": [
                [...],  // combined interval
                [...],  // interval 1
                ...
                [...]  // interval n
            ]
        },
        ...,
        
        
        "item_assets": {
            "item1": {},
            "item2": {},
        },
        "stac_version": "1.0.0",
        "stac_extensions": ['https://stac-extensions.github.io/item-assets/v1.0.0/schema.json'],
    }
}
```

**Important**: in eoapi, we are trying to set an example of good practice, so we choose to add `optional` extension in our collections. The `item_assets` extension is helpful because it quickly tells what assets are found within all the collection's items. This can help create visualizations for the whole collection without knowing much about its items.

In [5]:
collection = collections["collections"][0]
print(f"Collection Id: {collection['id']}")
print(f"Spatial Extent: {collection['extent']['spatial']['bbox'][0]}")
print(f"Temporal Extent: {collection['extent']['temporal']['interval'][0]}")

print(json.dumps(collections["collections"][0], indent=4))

Collection Id: MAXAR_BayofBengal_Cyclone_Mocha_May_23
Spatial Extent: [91.831615, 19.984656587012033, 92.97426268500965, 21.666101]
Temporal Extent: ['2023-01-03T04:30:17Z', '2023-03-14T04:30:25Z']
{
    "id": "MAXAR_BayofBengal_Cyclone_Mocha_May_23",
    "type": "Collection",
    "links": [
        {
            "rel": "items",
            "type": "application/geo+json",
            "href": "https://stac.eoapi.dev/collections/MAXAR_BayofBengal_Cyclone_Mocha_May_23/items"
        },
        {
            "rel": "parent",
            "type": "application/json",
            "href": "https://stac.eoapi.dev/"
        },
        {
            "rel": "root",
            "type": "application/json",
            "href": "https://stac.eoapi.dev/"
        },
        {
            "rel": "self",
            "type": "application/json",
            "href": "https://stac.eoapi.dev/collections/MAXAR_BayofBengal_Cyclone_Mocha_May_23"
        }
    ],
    "extent": {
        "spatial": {
            "bb

### Vizualizing the collections 

Each collection should have at least one spatial extent (combined), so let's use those to create a GeoJSON and add it to the map:

In [6]:
collections_geojson = {
    "type": "FeatureCollection",
    "features": [
        {
            'type': 'Feature',
            'geometry': {
                'type': 'Polygon',
                'coordinates': [[
                    [c["extent"]["spatial"]["bbox"][0][0], c["extent"]["spatial"]["bbox"][0][1]],
                    [c["extent"]["spatial"]["bbox"][0][2], c["extent"]["spatial"]["bbox"][0][1]],
                    [c["extent"]["spatial"]["bbox"][0][2], c["extent"]["spatial"]["bbox"][0][3]],
                    [c["extent"]["spatial"]["bbox"][0][0], c["extent"]["spatial"]["bbox"][0][3]],
                    [c["extent"]["spatial"]["bbox"][0][0], c["extent"]["spatial"]["bbox"][0][1]],
                ]]
            },
            'properties': {}
        }
        for c in collections["collections"]
    ]
}

m = Map(
    tiles="OpenStreetMap",
    location=(0, 0),
    zoom_start=1
)

geo_json = GeoJson(data=collections_geojson)
geo_json.add_to(m)
m

As mentioned, the collection's spatial extent can be a combination of multiple bbox, let's visualize all the spatial extent for one collection (`MAXAR_Maui_Hawaii_fires_Aug_23`)


Note: we will use the `/collections/{collectionId}` endpoint to get the collection metadata but it should be the same returned by the `/collections` endpoint.

Collection specification: https://github.com/radiantearth/stac-spec/blob/master/collection-spec/collection-spec.md

In [7]:
collection_id = "MAXAR_Maui_Hawaii_fires_Aug_23"

collection_info = httpx.get(f"https://stac.eoapi.dev/collections/{collection_id}").json()

geojson = {
    "type": "FeatureCollection",
    "features": [
        {
            'type': 'Feature',
            'geometry': {
                'type': 'Polygon',
                'coordinates': [[
                    [bbox[0], bbox[1]],
                    [bbox[2], bbox[1]],
                    [bbox[2], bbox[3]],
                    [bbox[0], bbox[3]],
                    [bbox[0], bbox[1]],
                ]]
            },
            'properties': {}
        }
        for bbox in collection_info["extent"]["spatial"]["bbox"]
    ]
}

mainbbox = collection_info["extent"]["spatial"]["bbox"][0]
m = Map(
    tiles="OpenStreetMap",
    location=((mainbbox[1] + mainbbox[3]) / 2,(mainbbox[0] + mainbbox[2]) / 2),
    zoom_start=9
)

geo_json = GeoJson(data=geojson)
geo_json.add_to(m)
m

#### Temporal Extent

Each collection can have spatial and temporal extents. As for the spatial extent, a collection can have multiple temporal extents but with its first one representing the combined min/max of all the intervals.

In [8]:
print(collection_info["extent"]["temporal"])

{'interval': [['2023-08-09T21:10:34Z', '2023-08-12T21:18:20Z']]}


## Items
Let's check the items for the Hawaii Fire collections: `MAXAR_Maui_Hawaii_fires_Aug_23`

In this section, we will:

- List all items for a specific collection using the `/collections/{collection_id}/items` endpoint
- Talk about the `limit` parameter
- Visualize all items on a map
- Talk about the item metadata
- List the Assets available for one Item

In [9]:
items = httpx.get(f"https://stac.eoapi.dev/collections/{collection_id}/items").json()
        
print(f"Nb Items in Db: {items['context']['matched']}")
print(f"Returned {len(items['features'])} Items")

Nb Items in Db: 123
Returned 10 Items


As you can see below, the `/items` endpoints returned only 10 items. To return more data we need to either use the `paging` mechanism or use the `limit` query parameter to get more items per request.

In [10]:
items = httpx.get(
    f"https://stac.eoapi.dev/collections/{collection_id}/items",
    params={
        "limit": 200,
    }
).json()

        
print(f"Nb Items in Db: {items['context']['matched']}")
print(f"Returned {len(items['features'])} Items")

Nb Items in Db: 123
Returned 123 Items


In [11]:
m = Map(
    tiles="OpenStreetMap",
    # Note: we re-use the `mainbbox` variable from previous cell
    location=((mainbbox[1] + mainbbox[3]) / 2,(mainbbox[0] + mainbbox[2]) / 2),
    zoom_start=9
)

geo_json = GeoJson(data=items)
geo_json.add_to(m)
m

#####  Item metadata 

Each item should have an `id`, a `geometry`, some links to `Assets` and a set of properties.

Item specification: https://github.com/radiantearth/stac-spec/blob/master/item-spec/item-spec.md

In [12]:
item = items["features"][0]
print("Item example:")
print(json.dumps(item, indent=4))

Item example:
{
    "id": "4_122000331020_105001003590C300",
    "bbox": [
        -156.31126031485115,
        20.82185619107944,
        -156.267548,
        20.846008751160078
    ],
    "type": "Feature",
    "links": [
        {
            "rel": "collection",
            "type": "application/json",
            "href": "https://stac.eoapi.dev/collections/MAXAR_Maui_Hawaii_fires_Aug_23"
        },
        {
            "rel": "parent",
            "type": "application/json",
            "href": "https://stac.eoapi.dev/collections/MAXAR_Maui_Hawaii_fires_Aug_23"
        },
        {
            "rel": "root",
            "type": "application/json",
            "href": "https://stac.eoapi.dev/"
        },
        {
            "rel": "self",
            "type": "application/geo+json",
            "href": "https://stac.eoapi.dev/collections/MAXAR_Maui_Hawaii_fires_Aug_23/items/4_122000331020_105001003590C300"
        }
    ],
    "assets": {
        "visual": {
            "href": "s

In [13]:
print("Item Id", item["id"])
print("Item Assets:", list(item["assets"].keys()))
print("Item Properties:")
print(json.dumps(item["properties"], indent=4))

Item Id 4_122000331020_105001003590C300
Item Assets: ['visual', 'data-mask', 'ms_analytic', 'pan_analytic']
Item Properties:
{
    "gsd": 0.5,
    "quadkey": "122000331020",
    "datetime": "2023-08-12T21:18:20Z",
    "platform": "GE01",
    "utm_zone": 4,
    "grid:code": "MXRA-Z4-122000331020",
    "proj:bbox": [
        779843.75,
        2304843.75,
        784374.5983184774,
        2307445.5041471347
    ],
    "proj:epsg": 32604,
    "catalog_id": "105001003590C300",
    "view:azimuth": 298.0,
    "proj:geometry": {
        "type": "Polygon",
        "coordinates": [
            [
                [
                    779843.75,
                    2304843.75
                ],
                [
                    779843.75,
                    2307445.5041471347
                ],
                [
                    784356.1192933423,
                    2307263.893928079
                ],
                [
                    784374.5983184774,
                    2304843.

#### Find acquisition times

By definition, every item should have either a `datetime` or a `start/end_datetime` property. For the Maxar dataset, we are assuming that `datetime` is acquisition times.

In [14]:
datetimes = {item["properties"]["datetime"] for item in items["features"]}
print("Dates:", sorted(list(datetimes)))

Dates: ['2023-08-09T21:10:34Z', '2023-08-09T21:10:35Z', '2023-08-09T21:20:30Z', '2023-08-09T21:20:31Z', '2023-08-12T21:11:16Z', '2023-08-12T21:11:17Z', '2023-08-12T21:12:48Z', '2023-08-12T21:12:49Z', '2023-08-12T21:18:17Z', '2023-08-12T21:18:18Z', '2023-08-12T21:18:19Z', '2023-08-12T21:18:20Z']


##### Other metadata

In [15]:
catalog_id = {item["properties"]["catalog_id"] for item in items["features"]}
print("Maxar Catalog Id:", sorted(list(catalog_id)))

platform = {item["properties"]["platform"] for item in items["features"]}
print("Maxar Platform: ", sorted(list(platform)))

Maxar Catalog Id: ['10300100EB15FF00', '10300100EB592000', '10300100EC8B7900', '105001003590C300', '10503F0006FE4B00']
Maxar Platform:  ['GE01', 'WV02']


Note: The `catalog_id` represents a unique acquisition ID. In our case, this means that all items are derived from 5 different acquisitions. 

> The unique identifier for the item, as assigned by the provider, Maxar. This is also referred to as the "acquisition ID".

ref: https://ard.maxar.com/docs/select-and-order/select-stac-collection-file/#properties-object


Let's look back at the item's datetime. If we remove the `seconds`:

In [16]:
datetimes = {item["properties"]["datetime"][:-4] for item in items["features"]}
print("Dates (without seconds):", sorted(list(datetimes)))

Dates (without seconds): ['2023-08-09T21:10', '2023-08-09T21:20', '2023-08-12T21:11', '2023-08-12T21:12', '2023-08-12T21:18']


We get 5 different dates, which might correspond to 5 different acquisitions (keep this in mind for later).

## Assets 

So we have **123** items for the `MAXAR_Maui_Hawaii_fires_Aug_23` collection, and each item has **4** assets (this is also found at the collection level in the `item_assets extension`).

In [17]:
print(json.dumps(item["assets"]["visual"], indent=4))

for name, asset in item["assets"].items():
    print(name, ": ", asset["type"])

{
    "href": "s3://maxar-opendata/events/Maui-Hawaii-fires-Aug-23/ard/04/122000331020/2023-08-12/105001003590C300-visual.tif",
    "type": "image/tiff; application=geotiff; profile=cloud-optimized",
    "roles": [
        "visual"
    ],
    "title": "Visual Image",
    "eo:bands": [
        {
            "name": "BAND_R",
            "common_name": "red",
            "description": "Red"
        },
        {
            "name": "BAND_G",
            "common_name": "green",
            "description": "Green"
        },
        {
            "name": "BAND_B",
            "common_name": "blue",
            "description": "Blue"
        }
    ],
    "alternate": {
        "public": {
            "href": "https://maxar-opendata.s3.amazonaws.com/events/Maui-Hawaii-fires-Aug-23/ard/04/122000331020/2023-08-12/105001003590C300-visual.tif",
            "title": "Public Access"
        }
    },
    "proj:bbox": [
        779843.75,
        2304843.75,
        785156.25,
        2310156.25
    ]

# Raster

In eoAPI, we have a raster API connected to the PgSTAC database. The service is built using [titiler-pgstac](http://github.com/stac-utils/titiler-pgstac) and can be used to visualize `Item` or `Mosaics` (multiple items).


Endpoint: https://raster.eoapi.dev


In this section, we will:

- Learn how to visualize Assets for one Item
- Create mosaic to visualize multiple Items 

## Asset visualization

We know we have 4 assets for each item, and 3 are of `Cloud-Optimized` type. Let's use the raster API to visualize them.


First, let's get the Raster metadata for each `raster` asset. The raster API will use the asset's `type` metadata to filter non-raster datasets (e.g.  `application/geopackage+sqlite3`).


In [18]:
# fetching Raster information for all the `raster` assets
item_id = item["id"]

print(f"Fetching Raster info for Item {item_id}")
info = httpx.get(f"https://raster.eoapi.dev/collections/{collection_id}/items/{item_id}/info").json()

Fetching Raster info for Item 4_122000331020_105001003590C300


In [19]:
print("Returned metadata for Assets:", list(info.keys()))

Returned metadata for Assets: ['visual', 'ms_analytic', 'pan_analytic']


In [20]:
print(json.dumps(info["visual"], indent=4))

{
    "bounds": [
        -156.31126031485115,
        20.821737319524217,
        -156.259392685322,
        20.870494972421625
    ],
    "minzoom": 12,
    "maxzoom": 19,
    "band_metadata": [
        [
            "b1",
            {}
        ],
        [
            "b2",
            {}
        ],
        [
            "b3",
            {}
        ]
    ],
    "band_descriptions": [
        [
            "b1",
            ""
        ],
        [
            "b2",
            ""
        ],
        [
            "b3",
            ""
        ]
    ],
    "dtype": "uint8",
    "nodata_type": "Mask",
    "colorinterp": [
        "red",
        "green",
        "blue"
    ],
    "driver": "GTiff",
    "count": 3,
    "width": 17408,
    "height": 17408,
    "overviews": [
        2,
        4,
        8,
        16,
        32,
        64
    ]
}


The `/collections/{collectionId}/items/{itemId}/info` endpoint returned metadata for 3 assets (the raster ones). We now know a bit more about each asset (datatype, zoom levels, number of bands), which can help us create tiles urls.


##### Asset on Map

To visualize an asset on a Map, we need to construct a `Tile URL`. To ease the task we can use the raster's service `/collections/{collection_id}/items/{item_id}/tilejson.json` endpoint, but here are the requirements:

- HAVE TO pass `assets` or `expression` parameter
- CAN pass `min/max zooms` (which will avoid under/over-zooming)
- CAN pass `rescale` parameter if datatype is not compatible with PNG/JPEG output format
- CAN pass `asset_bidx` parameter to select band combination

In [21]:
# `visual` Asset

tilejson = httpx.get(
    f"https://raster.eoapi.dev/collections/{collection_id}/items/{item_id}/tilejson.json",
    params = (
        ("assets", "visual"),  # THIS PARAMETER IS MANDATORY
        ("minzoom", 12),  # By default the tiler will use 0
        ("maxzoom", 19), # By default the tiler will use 24
    )
).json()
print(tilejson)

bounds = tilejson["bounds"]
m = Map(
    location=((bounds[1] + bounds[3]) / 2,(bounds[0] + bounds[2]) / 2),
    zoom_start=12
)

tiles = TileLayer(
    tiles=tilejson["tiles"][0],
    min_zoom=tilejson["minzoom"],
    max_zoom=tilejson["maxzoom"],
    opacity=1,
    attr="Maxar"
)
tiles.add_to(m)

geo_json = GeoJson(data=item)
geo_json.add_to(m)
m

{'tilejson': '2.2.0', 'version': '1.0.0', 'scheme': 'xyz', 'tiles': ['https://raster.eoapi.dev/collections/MAXAR_Maui_Hawaii_fires_Aug_23/items/4_122000331020_105001003590C300/tiles/WebMercatorQuad/{z}/{x}/{y}@1x?assets=visual'], 'minzoom': 12, 'maxzoom': 19, 'bounds': [-156.31126031485115, 20.82185619107944, -156.267548, 20.846008751160078], 'center': [-156.28940415742557, 20.83393247111976, 12]}


In [22]:
# `visual` Asset with different band combination

tilejson = httpx.get(
    f"https://raster.eoapi.dev/collections/{collection_id}/items/{item_id}/tilejson.json",
    params = (
        ("assets", "visual"),
        ("asset_bidx", "visual|3,2,1"),  # We use the `asset_bidx` parameter to change the band combination
        ("minzoom", 12),
        ("maxzoom", 19),
    )
).json()

bounds = tilejson["bounds"]
m = Map(
    location=((bounds[1] + bounds[3]) / 2,(bounds[0] + bounds[2]) / 2),
    zoom_start=14
)

tiles = TileLayer(
    tiles=tilejson["tiles"][0],
    min_zoom=tilejson["minzoom"],
    max_zoom=tilejson["maxzoom"],
    opacity=1,
    attr="Maxar"
)
tiles.add_to(m)

geo_json = GeoJson(data=item)
geo_json.add_to(m)
m

the `visual` asset is pretty easy to work with because it has only 3 bands and if of `uint8` datatype, meaning that we can translate the data directly to PNG or JPEG without needed to do any rescaling nor band selection. 

This is not always the case, let's use the `ms_analytic` asset now. 

This asset has more than 3 bands and is also of `Uint16` data type, so to be able to visualize it, we will need to:
- select 1 or 3 bands
- apply some linear rescaling between min/max values

In [23]:
print(json.dumps(info["ms_analytic"], indent=4))

{
    "bounds": [
        -156.31126031485115,
        20.821737319524217,
        -156.259392685322,
        20.870494972421625
    ],
    "minzoom": 12,
    "maxzoom": 16,
    "band_metadata": [
        [
            "b1",
            {}
        ],
        [
            "b2",
            {}
        ],
        [
            "b3",
            {}
        ],
        [
            "b4",
            {}
        ]
    ],
    "band_descriptions": [
        [
            "b1",
            ""
        ],
        [
            "b2",
            ""
        ],
        [
            "b3",
            ""
        ],
        [
            "b4",
            ""
        ]
    ],
    "dtype": "uint16",
    "nodata_type": "Mask",
    "colorinterp": [
        "gray",
        "undefined",
        "undefined",
        "undefined"
    ],
    "driver": "GTiff",
    "count": 4,
    "width": 2777,
    "height": 2777,
    "overviews": [
        2,
        4,
        8
    ]
}


To choose the `min/max` rescaling values wizely we can first use the `/statistics` endpoint to get an approximation of the dataset data range

In [24]:
stats = httpx.get(
    f"https://raster.eoapi.dev/collections/{collection_id}/items/{item_id}/statistics",
    params={"assets": "ms_analytic"}
).json()
print(json.dumps(stats, indent=4))

{
    "ms_analytic_b1": {
        "min": 15.0,
        "max": 9390.0,
        "mean": 218.3455215862242,
        "count": 422538.0,
        "sum": 92259280.0,
        "std": 221.38558325187785,
        "median": 150.0,
        "majority": 74.0,
        "minority": 15.0,
        "unique": 2537.0,
        "histogram": [
            [
                418845.0,
                2728.0,
                676.0,
                168.0,
                70.0,
                19.0,
                9.0,
                7.0,
                9.0,
                7.0
            ],
            [
                15.0,
                952.5,
                1890.0,
                2827.5,
                3765.0,
                4702.5,
                5640.0,
                6577.5,
                7515.0,
                8452.5,
                9390.0
            ]
        ],
        "valid_percent": 40.3,
        "masked_pixels": 626038.0,
        "valid_pixels": 422538.0,
        "percentile_2": 34.0,

Min/Max values for each band of the `ms_analytic` asset

In [25]:
for k, v in stats.items():
    print(k, "Min/Max:", v['min'], "-", v['max'], )

ms_analytic_b1 Min/Max: 15.0 - 9390.0
ms_analytic_b2 Min/Max: 11.0 - 9396.0
ms_analytic_b3 Min/Max: 9.0 - 9442.0
ms_analytic_b4 Min/Max: 48.0 - 7906.0


In [26]:
# `ms_analytic` 
tilejson = httpx.get(
    f"https://raster.eoapi.dev/collections/{collection_id}/items/{item_id}/tilejson.json",
    params = (
        ("assets", "ms_analytic"),
        ("asset_bidx", "ms_analytic|4,1,2"),
        ("rescale", "0,8000"),  # Here we use min/max found in the statistics response
        ("minzoom", 12),
        ("maxzoom", 16), 
    )
).json()

bounds = tilejson["bounds"]
m = Map(
    location=((bounds[1] + bounds[3]) / 2,(bounds[0] + bounds[2]) / 2),
    zoom_start=14
)

tiles = TileLayer(
    tiles=tilejson["tiles"][0],
    min_zoom=tilejson["minzoom"],
    max_zoom=tilejson["maxzoom"],
    opacity=1,
    attr="Maxar"
)
tiles.add_to(m)

geo_json = GeoJson(data=item)
geo_json.add_to(m)
m

See full list of options at https://stac-utils.github.io/titiler-pgstac/item_endpoints/#tiles

## Mosaics

As mentioned and shown with `titiler-pgstac`, we can visualize an item's asset, but the real power of the raster API is  to be able to create a virtual mosaic dynamically and merge multiple items on the fly.

Learn more: http://github.com/stac-utils/titiler-pgstac.

Let's create `mosaics` for a specific DateTime we found in the previous step.

> ['2023-08-09T21:10', '2023-08-09T21:20', '2023-08-12T21:11', '2023-08-12T21:12', '2023-08-12T21:18']

In [27]:
# Create a Mosaic for the first date `2023-08-09`
startdate = "2023-08-09T00:00:01Z"
enddate = "2023-08-10T23:59:59Z"

mosaic = httpx.post(
    "https://raster.eoapi.dev/searches/register",
    data=json.dumps(
        {
            "filter-lang": 'cql2-json',
            "filter": {
                "op": 'and', 
                "args": [
                    {
                        "op": "in", 
                        "args": [{"property": "collection"}, [collection_id]]
                    },
                    {
                        "op": "t_intersects", 
                        "args": [
                            {"property": "datetime"}, [startdate, enddate]
                        ]
                    }
                ],
            },
            "metadata":{
                "bounds": collection_info["extent"]["spatial"]["bbox"][0],
            }
            
        }
    )
).json()
print(json.dumps(mosaic, indent=4))

{
    "id": "c13976bc1e5dc7f3c2b4a64239871fc0",
    "links": [
        {
            "rel": "metadata",
            "title": "Mosaic metadata",
            "type": "application/json",
            "href": "https://raster.eoapi.dev/searches/c13976bc1e5dc7f3c2b4a64239871fc0/info"
        },
        {
            "rel": "tilejson",
            "title": "Link for TileJSON",
            "type": "application/json",
            "href": "https://raster.eoapi.dev/searches/c13976bc1e5dc7f3c2b4a64239871fc0/tilejson.json"
        },
        {
            "rel": "map",
            "title": "Link for Map viewer",
            "type": "application/json",
            "href": "https://raster.eoapi.dev/searches/c13976bc1e5dc7f3c2b4a64239871fc0/map"
        },
        {
            "rel": "wmts",
            "title": "Link for WMTS",
            "type": "application/json",
            "href": "https://raster.eoapi.dev/searches/c13976bc1e5dc7f3c2b4a64239871fc0/WMTSCapabilities.xml"
        }
    ]
}


Explanation:

The `mosaic` correspond of a STAC Search query. Because we use `PgSTAC` backend we can make use of the `filter` extension to construct complex query: https://github.com/stac-api-extensions/filter


```json
{
    // PgSTAC accepts multiple languages for filtering; here we will use cql2-json
    "filter-lang": 'cql2-json',
    // We tell PgSTAC to `register` a `search` request with the following filter:
    "filter": {
        "op": 'and', 
        "args": [
            // Item's collection HAS TO be in `[collection_id]`
            {
                "op": "in", 
                "args": [{"property": "collection"}, [collection_id]]
            },
            {
            // Item's datetime property HAS to intersect with the start/end date
                "op": "t_intersects", 
                "args": [
                    {"property": "datetime"}, [startdate, enddate]
                ]
            },
            // titiler-pgstac accept some additional metadata
            // https://stac-utils.github.io/titiler-pgstac/advanced/metadata/
            // One is useful: `bounds`. When creating a mosaic, the tiler will have no idea 
            // where the items will be before trying to create each tile. To avoid trying to request 
            // tiles where we know we don't have any items, we can add the collection's extent to the 
            // mosaic metadata. 
            // The tiler service will then return the bounds in the tilejson document for the 
            // client application.
            "metadata":{
                "bounds": collection_info["extent"]["spatial"]["bbox"][0],
            }            
        ],
    },

}
```

API Response:

The raster service will return a `searchid` hash (also called `mosaic id`), which we can use to construct a tile URL.

To create a valid tile URL we will again need to pass an `assets` parameter to tell the tiler which assets we want to visualize. We can also set the min/max zooms limits to avoid underzooming (openning to many files) and overzooming.

See full list of options: https://stac-utils.github.io/titiler-pgstac/mosaic_endpoints/#tiles

In [32]:
mosaic_id = mosaic["id"]

tilejson = httpx.get(
    f"https://raster.eoapi.dev/searches/{mosaic_id}/tilejson.json",
    params = (
        ("assets", "visual"),  # THIS IS MANDATORY
        ("minzoom", 12),
        ("maxzoom", 19), 
    )
).json()
print(tilejson)

bounds = tilejson["bounds"]
m = Map(
    location=((bounds[1] + bounds[3]) / 2,(bounds[0] + bounds[2]) / 2),
    zoom_start=12
)

tiles = TileLayer(
    tiles=tilejson["tiles"][0],
    min_zoom=tilejson["minzoom"],
    max_zoom=tilejson["maxzoom"],
    opacity=1,
    attr="Maxar"
)
tiles.add_to(m)
m

{'tilejson': '2.2.0', 'name': 'db5a7f4dc6d5763bf955e80903e6a78d', 'version': '1.0.0', 'scheme': 'xyz', 'tiles': ['https://raster.eoapi.dev/searches/db5a7f4dc6d5763bf955e80903e6a78d/tiles/WebMercatorQuad/{z}/{x}/{y}?assets=visual'], 'minzoom': 12, 'maxzoom': 19, 'bounds': [-180.0, -85.05112877980655, 179.99999999999955, 85.0511287798066], 'center': [-2.2737367544323206e-13, 2.842170943040401e-14, 12]}


### More with Mosaic

Let's get crazy and create a `multiple-collection` mosaic with all items which have the `GE01` plateform.

First check how many items have the platform="GE01" usin the stac `/search` endpoint

In [ ]:
items = httpx.post(
    "https://stac.eoapi.dev/search",
    data=json.dumps(
        {
            "filter-lang": 'cql2-json',
            "filter": {
                "op": 'and', 
                "args": [
                    {
                        "op": "eq", 
                        "args": [
                            {"property": "platform"}, "GE01"
                        ]
                    }
                ],
            },
            "limit": 1,
        }
    )
).json()

print(f"Found {items['context']['matched']} Items matching the query")

Found 2246 Items matching the query


Woot, more than 2000 items.

Let's now register the mosaic query.

Note, for this we will assume:

- that `GE01` is a cross-collection platform
- that all `GE01` items have a `visual` asset
- that all `visual` assets are 3 bands and Uint8 raster
- that all `visual` assets cover the same zoom range

In [30]:
mosaic = httpx.post(
    "https://raster.eoapi.dev/searches/register",
    data=json.dumps(
        {
            "filter-lang": 'cql2-json',
            "filter": {
                "op": 'and', 
                "args": [
                    {
                        "op": "eq", 
                        "args": [
                            {"property": "platform"}, "GE01"
                        ]
                    }
                ],
            },
            "metadata":{}
            
        }
    )
).json()

mosaic_id = mosaic["id"]

tilejson = httpx.get(
    f"https://raster.eoapi.dev/searches/{mosaic_id}/tilejson.json",
    params = (
        ("assets", "visual"),  # THIS IS MANDATORY
        ("minzoom", 12),
        ("maxzoom", 19), 
    )
).json()
print(tilejson)

bounds = tilejson["bounds"]
m = Map(
    location=(0, 0),
    zoom_start=8
)

# let's add the collection's geojson on top of the map
geo_json = GeoJson(data=collections_geojson)
geo_json.add_to(m)

tiles = TileLayer(
    tiles=tilejson["tiles"][0],
    min_zoom=tilejson["minzoom"],
    max_zoom=tilejson["maxzoom"],
    opacity=1,
    attr="Maxar"
)
tiles.add_to(m)
m



{'tilejson': '2.2.0', 'name': 'db5a7f4dc6d5763bf955e80903e6a78d', 'version': '1.0.0', 'scheme': 'xyz', 'tiles': ['https://raster.eoapi.dev/searches/db5a7f4dc6d5763bf955e80903e6a78d/tiles/WebMercatorQuad/{z}/{x}/{y}?assets=visual'], 'minzoom': 12, 'maxzoom': 19, 'bounds': [-180.0, -85.05112877980655, 179.99999999999955, 85.0511287798066], 'center': [-2.2737367544323206e-13, 2.842170943040401e-14, 12]}


Not all collection have GE01 items; pan/zoom to some collections to see the tiles.

Note: While we showed that it is technically possible to construct Mosaics using the `open` filter, there might be better ideas in terms of performance.

- Only some properties are indexed in PgSTAC, meaning the `platform` properties lookup will be slow.
- We cannot spatially limit the mosaic, meaning the frontend will request tiles for the entire world.
- Assuming that all `GE01` platforms have `visual` assets is a HUGE assumption.
- Assuming that all `visual` assets are the same is also bold.

## What's Next?

### Spin Up Your Own eoAPI Instance

You've seen what eoAPI can do with Maxar data in the context of the Hawaii wildfires. Interested in setting up your own eoAPI service? It's straightforward! Follow the 'Getting Started' section of the [eoAPI GitHub repository](https://github.com/developmentseed/eoAPI) to get your instance up and running. This will give you greater control and customization options.

### Contribute Your Data

Consider contributing if you've used Maxar data for similar analyses or have other datasets that could benefit the community. Uploading your data to your eoAPI instance can provide more diverse examples and help in various applications.

### Community-Driven Examples

The examples you see here, including this notebook, are often community-driven. We encourage you to contribute your analyses, workflows, or visualizations. Your insights could be invaluable for helping others.

### Have Questions? Join the Discussion!

If you have questions, feedback, or want to engage with the eoAPI community, please join the [GitHub discussions](https://github.com/developmentseed/eoAPI/discussions). It's a great place to ask questions, share your experiences, and connect with others interested in eoAPI and its components. 

---

Thank you for taking the time to go through this notebook.